
A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [4]:
import pandas as pd
import numpy as np

data=pd.read_csv('rental_info.csv')
pd.set_option('display.max_columns', 20)
data.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [5]:
data['rental_date'] = pd.to_datetime(data['rental_date'])
data['return_date'] = pd.to_datetime(data['return_date'])
data['rental_duration'] = (data['return_date'] - data['rental_date']).dt.days
data

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_duration
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15856,2005-08-22 10:49:15+00:00,2005-08-29 09:52:15+00:00,6.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,48.8601,7744.0,24.9001,6
15857,2005-07-31 09:48:49+00:00,2005-08-04 10:53:49+00:00,4.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,24.9001,7744.0,24.9001,4
15858,2005-08-20 10:35:30+00:00,2005-08-29 13:03:30+00:00,8.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,80.8201,7744.0,24.9001,9
15859,2005-07-31 13:10:20+00:00,2005-08-08 14:07:20+00:00,7.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,63.8401,7744.0,24.9001,8


In [6]:
data.isnull().count()

rental_date         15861
return_date         15861
amount              15861
release_year        15861
rental_rate         15861
length              15861
replacement_cost    15861
special_features    15861
NC-17               15861
PG                  15861
PG-13               15861
R                   15861
amount_2            15861
length_2            15861
rental_rate_2       15861
rental_duration     15861
dtype: int64

In [7]:
data.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
15856    False
15857    False
15858    False
15859    False
15860    False
Length: 15861, dtype: bool

In [8]:
X = data.drop(['rental_duration','rental_date','return_date','amount_2','length_2','rental_rate_2'], axis=1).values
y = data['rental_duration'].values

In [9]:
X

array([[2.99, 2005.0, 2.99, ..., 0, 0, 1],
       [2.99, 2005.0, 2.99, ..., 0, 0, 1],
       [2.99, 2005.0, 2.99, ..., 0, 0, 1],
       ...,
       [8.99, 2009.0, 4.99, ..., 0, 0, 1],
       [7.99, 2009.0, 4.99, ..., 0, 0, 1],
       [5.99, 2009.0, 4.99, ..., 0, 0, 1]], dtype=object)

In [10]:
y

array([3, 2, 7, ..., 9, 8, 6], dtype=int64)

In [20]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,5] = le.fit_transform(X[:,5] )

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
print("Trainig Data..")
print("The shape of training features: ", X_train.shape)
print("The shape of training labels: ", y_train.shape)
print("Testing Data..")
print("The shape of testing features: ", X_test.shape)
print("The shape of testing labels: ", y_test.shape)

Trainig Data..
The shape of training features:  (12688, 10)
The shape of training labels:  (12688,)
Testing Data..
The shape of testing features:  (3173, 10)
The shape of testing labels:  (3173,)


In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
training_acc = np.ceil(model.score(X_train, y_train) * 100)
print("The Training Accuracy = ", training_acc, "%")

The Training Accuracy =  59.0 %


In [15]:
model.fit(X_test, y_test)
y_pred = model.predict(X_test)
testing_acc = np.ceil(model.score(X_test, y_test) * 100)
print("The Testing Accuracy = ", testing_acc, "%")

The Testing Accuracy =  57.0 %


In [16]:
print(model.coef_)
print(model.intercept_)

[ 1.17688831e+00  2.02674251e-02 -1.17624776e+00  1.63440282e-03
 -3.81089496e-04 -7.37592949e-03  3.46178277e-01  1.13337694e-01
  3.42810999e-01  9.64095472e-02]
-37.92521047871047


In [17]:
import sklearn.metrics as sm
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, y_pred), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(y_test, y_pred), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(y_test, y_pred), 2)) 
print("R2 score =", round(sm.r2_score(y_test, y_pred), 2))

Mean absolute error = 1.44
Mean squared error = 2.97
Explain variance score = 0.57
R2 score = 0.57


In [26]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=4)
X_train_poly = poly.fit_transform(X_train)

poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)

y_pred_train = poly_model.predict(X_train_poly)

training_acc = np.ceil(poly_model.score(X_train_poly, y_train) * 100)
print("The Training Accuracy = ", training_acc, "%")

The Training Accuracy =  70.0 %


In [27]:
poly = PolynomialFeatures(degree=4)
X_test_poly = poly.fit_transform(X_test)

poly_model = LinearRegression()
poly_model.fit(X_test_poly, y_test)

y_pred_test = poly_model.predict(X_test_poly)

test_acc = np.ceil(poly_model.score(X_test_poly, y_test) * 100)
print("The Testing Accuracy = ", test_acc, "%")

The Testing Accuracy =  73.0 %
